In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pathlib as plib
import sys
import time
import utils
import cp_optimizer
import math

from ortools.sat.python import cp_model

In [43]:
q_list = ["q1", "q3", "q2", "q4","q2", "q5", "q6"]
df_queries = utils.load(q_list, num_partitions='16')
res = utils.interpolate(df_queries)
precision = 1000
C =16
C_ = None # for split
Q = len(q_list)
R = int(round(Q / 2))
T = cp_optimizer.init_matrix(q_list, res, C, precision)
reg_factor = 0.001
T_arr = np.array(T)
T_min = np.min(T_arr[np.nonzero(T_arr)])
T_min = 0

In [44]:
def print_tree(q_list, probas):
    height = int(math.log2(len(q_list)))
    
    proba_str = ""
    for p in probas:
        proba_str += str(round(p, 6)) + "   "
    l = len(proba_str)
    print(" "*int(l/2) + q_list[0] + "\n")
    
    for i in range(height):
        blank = " "*int(l/(2**(i + 1) + 1))
        s = blank
        for j in range(2**i):
            idx_left = 2**(i + 1) + j*2 - 1
            s += q_list[idx_left] + blank
            s += q_list[idx_left + 1] + blank
        print(s + "\n")
    
    print(" "*int(l/(2**(i + 2) + 1)) + proba_str)
            

In [45]:

path_sets_idx = utils.get_path_sets(range(len(q_list)))
num_paths = len(path_sets_idx)
probas = np.random.random(size=num_paths)
probas = list(probas / sum(probas))
path_sets_idx

[{0, 1, 3}, {0, 1, 4}, {0, 2, 5}, {0, 2, 6}]

In [46]:
print_tree(q_list, probas)

                      q1

              q3              q2              

        q4        q2        q5        q6        

    0.524541   0.257837   0.038343   0.179279   


## 1st formulation

In [47]:
#Define program
model = cp_model.CpModel()
#Init Variable
V = {(q, r): model.NewIntVar(0, C, f'V_{q},{r}') for q in range(Q) for r in range(R)}


I = {(q, r): model.NewIntVar(0, R - 1, f'I_{q},{r}') for q in range(Q) for r in range(R)}
X = {q: model.NewIntVar(1, C, f'X_{q}') for q in range(Q)}
k = {r: model.NewIntVar(0, T.max(), f'k_{r}') for r in range(R)}
t_ind = {(q, r): model.NewIntVar(0, T.max(), f't_ind_{q},{r}') for q in range(Q) for r in range(R)}

A = np.zeros((Q, R)).astype(int).astype(object)
for q in range(Q):
    A[q, -1] = X[q]
#V, I, X, k, t_ind, A

#for probability approach
V_bool = {(q, r): model.NewBoolVar(f'V_bool_{q},{r}') for q in range(Q) for r in range(R)}
index_run = {q: model.NewIntVar(0, R - 1, f'index_run_{q}') for q in range(Q)}
#TODO :check if can reduce range
runtime_runs = {r: model.NewIntVar(T_min, T.max() * Q, f'run_r_{r}') for r in range(R)}

runtime_queries = {q: model.NewIntVar(T_min, T.max() * Q, f'run_q_{q}') for q in range(Q)}
runtime_paths = {p: model.NewIntVar(T_min, T.max() * Q, f'run_path_{q}') for p in range(num_paths)}

In [48]:
#Define program
# 1
for q in range(Q):
    for r in range(R):
        model.AddElement(I[q, r], list(A[q, :]), V[q, r])
    model.AddAllDifferent([I[q, r] for r in range(R)])
# 2
for r in range(R):
    if C_ is None:
        model.Add(sum(V[q, r] for q in range(Q)) <= C)
    else:
        model.Add(sum(V[q, r] for q in range(Q)) == C_)
# 3
for q in range(Q):
    for r in range(R):
        model.AddElement(V[q, r], list(T[q, :]), t_ind[q, r])
for r in range(R):
    model.AddMaxEquality(k[r], [t_ind[q, r] for q in range(Q)])

if probas is None:
    obj = sum(k[r] for r in range(R))
else :
    #Get index of which run each query is
    for q in range(Q):
        for r in range(R):
            model.Add(V[q,r] > 0).OnlyEnforceIf(V_bool[q,r])
            model.Add(V[q,r] == 0).OnlyEnforceIf(V_bool[q,r].Not())

    for q in range(Q):
        model.Add(index_run[q]==0).OnlyEnforceIf([V_bool[q,r].Not() for r in range(R)]) 
        for r in range(R):
            model.Add(index_run[q]== r).OnlyEnforceIf(V_bool[q,r]) 
            
    #Get runtime of each run
    for r in range(R):
        model.Add(runtime_runs[r] == sum(k[rr] for rr in range(r + 1)))
        
    #Get runtime of each query
    for q in range(Q):
        model.AddElement(index_run[q], runtime_runs, runtime_queries[q])
    #Set path runtime
    for (id_p, path_set) in enumerate(path_sets_idx):
        model.AddMaxEquality(runtime_paths[id_p], [runtime_queries[q] for q in path_set])
        
    obj = sum(probas[p] * runtime_paths[p] for p in range(num_paths)) +  reg_factor *sum(k[r] for r in range(R))

model.Minimize(obj)

In [49]:
start_time = time.time()
solver = cp_model.CpSolver()
status = solver.Solve(model)
process_time = time.time() - start_time
print(cp_optimizer.MODEL_STATUS[status])
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # (V, _, _, k, _, _) = variables
    runtime, res_schedule = cp_optimizer.model_to_solution(solver, R, V, k, q_list, precision)
else:
    runtime, res_schedule = -1, []

process_time, runtime, res_schedule

OPTIMAL


(1.0899319648742676,
 40.029,
 [[('q1', 2), ('q3', 4), ('q4', 2), ('q6', 8)],
  [('q2', 8), ('q2', 8)],
  [('q5', 16)]])

In [50]:
proba_variables = cp_optimizer.get_proba_variables(q_list, probas)
process_time, var_results = cp_optimizer.optimize(q_list, res, C, R, precision, proba_variables=proba_variables, reg_factor=reg_factor)

In [51]:
cp_optimizer.get_program_results(*var_results, True)

(40.029,
 [[('q1', 2), ('q3', 4), ('q4', 2), ('q6', 8)],
  [('q2', 8), ('q2', 8)],
  [('q5', 16)]])

## Second formulation test

In [52]:
#Define program
model = cp_model.CpModel()
#Init Variable
V = {(q, r): model.NewIntVar(0, C, f'V_{q},{r}') for q in range(Q) for r in range(R)}


I = {(q, r): model.NewIntVar(0, R - 1, f'I_{q},{r}') for q in range(Q) for r in range(R)}
X = {q: model.NewIntVar(1, C, f'X_{q}') for q in range(Q)}
k = {r: model.NewIntVar(0, T.max(), f'k_{r}') for r in range(R)}
t_ind = {(q, r): model.NewIntVar(0, T.max(), f't_ind_{q},{r}') for q in range(Q) for r in range(R)}

A = np.zeros((Q, R)).astype(int).astype(object)
for q in range(Q):
    A[q, -1] = X[q]
#V, I, X, k, t_ind, A

#for probability approach
V_bool = {(q, r): model.NewBoolVar(f'V_bool_{q},{r}') for q in range(Q) for r in range(R)}
index_run_q = {q: model.NewIntVar(0, R - 1, f'index_run_{q}') for q in range(Q)}
index_run_path = {p: model.NewIntVar(0, R - 1, f'index_run_{q}') for p in range(num_paths)}
#TODO :check if can reduce range
runtime_runs = {r: model.NewIntVar(T_min, T.max() * Q, f'run_r_{r}') for r in range(R)}

runtime_paths = {p: model.NewIntVar(T_min, T.max() * Q, f'run_path_{q}') for p in range(num_paths)}

In [53]:
#Define program
# 1
for q in range(Q):
    for r in range(R):
        model.AddElement(I[q, r], list(A[q, :]), V[q, r])
    model.AddAllDifferent([I[q, r] for r in range(R)])
# 2
for r in range(R):
    if C_ is None:
        model.Add(sum(V[q, r] for q in range(Q)) <= C)
    else:
        model.Add(sum(V[q, r] for q in range(Q)) == C_)
# 3
for q in range(Q):
    for r in range(R):
        model.AddElement(V[q, r], list(T[q, :]), t_ind[q, r])
for r in range(R):
    model.AddMaxEquality(k[r], [t_ind[q, r] for q in range(Q)])

if probas is None:
    obj = sum(k[r] for r in range(R))
else :
    #Get index of which run each query is
    for q in range(Q):
        for r in range(R):
            model.Add(V[q,r] > 0).OnlyEnforceIf(V_bool[q,r])
            model.Add(V[q,r] == 0).OnlyEnforceIf(V_bool[q,r].Not())

    for q in range(Q):
        model.Add(index_run_q[q]==0).OnlyEnforceIf([V_bool[q,r].Not() for r in range(R)]) 
        for r in range(R):
            model.Add(index_run_q[q]== r).OnlyEnforceIf(V_bool[q,r]) 
            
    #Get runtime of each run
    for r in range(R):
        model.Add(runtime_runs[r] == sum(k[rr] for rr in range(r + 1)))
        
    #Get index path
    for (p, path_set) in enumerate(path_sets_idx):
        model.AddMaxEquality(index_run_path[p], [index_run_q[q] for q in path_set])
        
    #Get runtime path
    for p in range(num_paths):
        model.AddElement(index_run_path[p], runtime_runs, runtime_paths[p])
        
    obj = sum(probas[p] * runtime_paths[p] for p in range(num_paths)) +  reg_factor *sum(k[r] for r in range(R))

model.Minimize(obj)

In [54]:
start_time = time.time()
solver = cp_model.CpSolver()
status = solver.Solve(model)
process_time = time.time() - start_time
print(cp_optimizer.MODEL_STATUS[status])
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # (V, _, _, k, _, _) = variables
    runtime, res_schedule = cp_optimizer.model_to_solution(solver, R, V, k, q_list, precision)
else:
    runtime, res_schedule = -1, []

process_time, runtime, res_schedule

OPTIMAL


(1.0041992664337158,
 40.029,
 [[('q1', 2), ('q3', 4), ('q4', 2), ('q6', 8)],
  [('q2', 8), ('q2', 8)],
  [('q5', 16)]])

In [14]:
[cp_model.UNKNOWN, cp_model.MODEL_INVALID,   cp_model.FEASIBLE, cp_model.INFEASIBLE, cp_model.OPTIMAL]

[0, 1, 2, 3, 4]

## Split proba test

In [15]:
print_tree(q_list, probas)

                     q1

              q3              q2              

        q4        q2        q5        q6        

    0.23771   0.326808   0.162884   0.272597   


In [16]:
proba_var = cp_optimizer.get_proba_variables(q_list, probas)
R_ = 2
C_ = 2 * len(q_list)
print(C_)
process_time, var_results = cp_optimizer.optimize(q_list, res, C, R=R_, precision=precision, 
                                                  C_=C_,
                                                  proba_variables=proba_var, reg_factor=0.0003
                                                 )
cp_optimizer.get_program_results(*var_results, True)

14


(46.923,
 [[('q1', 2), ('q2', 3), ('q4', 2), ('q2', 3), ('q5', 4)],
  [('q3', 4), ('q6', 10)]])

In [17]:

probas, num_paths, path_sets_idx = proba_var
_, res_schedule = cp_optimizer.get_program_results(*var_results, False)



In [18]:
left_queries, _ = zip(*res_schedule[0])
right_queries, _ = zip(*res_schedule[1])
left_queries = set(left_queries)
right_queries = set(right_queries)

# 1. split paths : determine which belong only to left
left_paths = []
right_paths_raw = []
for p in path_sets_idx:
    intersect = p.intersection(right_queries)
    if len(intersect) == 0:
        left_paths.append(p)
    else :
        right_paths_raw.append(p)

# 2. select queries appearing only in right paths
remain_queries = left_queries.intersection(set.union(*left_paths))
left_queries = left_queries.difference(remain_queries)
right_queries = right_queries.union(remain_queries)

# 3. redefine paths right containing queries in left
right_paths = []
for p in right_paths_raw:
    new_p = right_queries.intersection(p)
    right_paths.append(new_p)

# 4. redefine left and right q_list
q_list_left = [q_list[i] for i in sorted(left_queries)]
q_list_right = [q_list[i] for i in sorted(right_queries)]

# Note : need to keep left_queries and right_queries to reconstruct identically